<a href="https://colab.research.google.com/github/Vaishali9522/sample-project/blob/master/NeuralNeywork.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np

In [0]:
nn_architecture = [
                   {'input_dim': 400, 'output_dim': 4, 'activation': 'relu'},
                   {'input_dim': 4, 'output_dim': 6, 'activation': 'relu'},
                   {'input_dim': 6, 'output_dim': 6, 'activation': 'relu'},
                   {'input_dim': 6, 'output_dim': 4, 'activation': 'relu'},
                   {'input_dim': 4, 'output_dim': 1, 'activation': 'sigmoid'}
]

In [0]:
def init_layers(nn_architecure, seed=0):
  np.random.seed(seed)
  number_of_layers = len(nn_architecture)
  param_values = {}

  for idx, layer in enumerate(nn_architecture):
    layer_idx = idx + 1

    layer_input_size = layer['input_dim']
    layer_output_size = layer['output_dim']

    param_values['W' + str(layer_idx)] = np.random.randn(layer_output_size, layer_input_size) * 0.1
    param_values['b' + str(layer_idx)] = np.random.randn(layer_output_size, 1) * 0.1
  
  return param_values


In [0]:
def sigmoid(Z):
  return 1/(1 + np.exp(-Z))

def relu(Z):
  return np.maximum(0, Z)

def sigmoid_backward(dA, Z):
  sig = sigmoid(Z)
  return dA * sig * (1 - sig)

def relu_backward(dA, Z):
  dZ = np.array(dA, copy=True)
  dZ[Z <= 0] = 0
  return dZ

In [0]:
def single_layer_forward_propagation(A_prev, W_curr, b_curr, activation='relu'):
  Z_curr = np.dot(W_curr, A_prev) + b_curr

  if activation == 'relu':
    activation_func = relu
  elif activation == 'sigmoid':
    activation_func = sigmoid
  else:
    raise Exception('Non supported activation function')
  
  return activation_func(Z_curr), Z_curr

In [0]:
def full_forward_propagation(X, param_values, nn_architecture):
  memory = {}
  A_curr = X

  for idx, layer in enumerate(nn_architecture):
    layer_idx = idx + 1
    A_prev = A_curr

    activation_func_curr = layer['activation']
    W_curr = param_values['W' + str(layer_idx)]
    b_curr = param_values['b' + str(layer_idx)]

    A_curr, Z_curr = single_layer_forward_propagation(A_prev, W_curr, b_curr, activation_func_curr)

    memory['A' + str(idx)] = A_prev
    memory['Z' + str(layer_idx)] = Z_curr
  
  return A_curr, memory

In [0]:
def get_cost_value(Y_hat, Y):
    m = Y_hat.shape[1]
    print(m)
    print(Y_hat.shape, Y.shape)
    cost = (1/m) * np.sum(-Y*np.log(Y_hat) - (1-Y)*np.log(1-Y_hat))
    return np.squeeze(cost)


In [0]:
def single_layer_backward_propagation(dA_curr, W_curr, b_curr, Z_curr, A_prev, activation="relu"):
    m = A_prev.shape[1]
    
    if activation is "relu":
        backward_activation_func = relu_backward
    elif activation is "sigmoid":
        backward_activation_func = sigmoid_backward
    else:
        raise Exception('Non-supported activation function')
    
    dZ_curr = backward_activation_func(dA_curr, Z_curr)
    dW_curr = np.dot(dZ_curr, A_prev.T) / m
    db_curr = np.sum(dZ_curr, axis=1, keepdims=True) / m
    dA_prev = np.dot(W_curr.T, dZ_curr)

    return dA_prev, dW_curr, db_curr

In [0]:
def full_backward_propagation(Y_hat, Y, memory, param_values, nn_architecture):
  grads_values = {}
  m = Y.shape[1]
  Y = Y.reshape(Y_hat.shape)
  
  dA_prev = - (np.divide(Y, Y_hat) - np.divide(1 - Y, 1 - Y_hat));
  
  for layer_idx_prev, layer in reversed(list(enumerate(nn_architecture))):
      layer_idx_curr = layer_idx_prev + 1
      activ_function_curr = layer["activation"]
      
      dA_curr = dA_prev
      
      A_prev = memory["A" + str(layer_idx_prev)]
      Z_curr = memory["Z" + str(layer_idx_curr)]
      W_curr = params_values["W" + str(layer_idx_curr)]
      b_curr = params_values["b" + str(layer_idx_curr)]
      
      dA_prev, dW_curr, db_curr = single_layer_backward_propagation(
          dA_curr, W_curr, b_curr, Z_curr, A_prev, activ_function_curr)
      
      grads_values["dW" + str(layer_idx_curr)] = dW_curr
      grads_values["db" + str(layer_idx_curr)] = db_curr
  
  return grads_values

In [0]:
def update(params_values, grads_values, nn_architecture, learning_rate):
    for layer_idx, layer in enumerate(nn_architecture):
        params_values["W" + str(layer_idx)] -= learning_rate * grads_values["dW" + str(layer_idx)]        
        params_values["b" + str(layer_idx)] -= learning_rate * grads_values["db" + str(layer_idx)]

    return params_values

In [0]:
def train(X, Y, nn_architecture, epochs, learning_rate):
    params_values = init_layers(nn_architecture, 2)
    cost_history = []
    
    for i in range(epochs):
        Y_hat, cache = full_forward_propagation(X, params_values, nn_architecture)
        cost = get_cost_value(Y_hat, Y)
        cost_history.append(cost)
        
        
        grads_values = full_backward_propagation(Y_hat, Y, cashe, params_values, nn_architecture)
        params_values = update(params_values, grads_values, nn_architecture, learning_rate)
        
    return params_values, cost_history
